In [16]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.core import Environment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [17]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'c46a9435-c957-4e6c-a0f4-b9a597984773'
resource_group = 'mlops'
workspace_name = 'mlopsdev'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='cocotrain2017_20220424_180733')
dataset.to_pandas_dataframe()

,image_url,label,label_confidence,image_height,image_width
0,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'Person', 'isCrowd': False, 'polygo...",[1.0],640,425
1,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'Person', 'isCrowd': False, 'polygo...","[1.0, 1.0]",480,640
2,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'food', 'isCrowd': False, 'polygon'...",[1.0],425,640
3,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'animals', 'isCrowd': False, 'polyg...",[1.0],480,640
4,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'animals', 'isCrowd': False, 'polyg...","[1.0, 1.0]",457,640
...,...,...,...,...,...
400,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'Person', 'isCrowd': False, 'polygo...",[1.0],427,640
401,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'Person', 'isCrowd': False, 'polygo...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",427,640
402,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'Person', 'isCrowd': False, 'polygo...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",480,640
403,workspaceblobstore/cocotrain2017/train2017/000...,"[{'label': 'animals', 'isCrowd': False, 'polyg...","[1.0, 1.0, 1.0, 1.0, 1.0]",427,640


In [18]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: mlopsdev
Azure region: centralus
Subscription id: c46a9435-c957-4e6c-a0f4-b9a597984773
Resource group: mlops


In [19]:
from azureml.train.automl import AutoMLImageConfig
from azureml.automl.core.shared.constants import ImageTask


In [20]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6s_V3', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-04-19T19:44:25.541000+00:00', 'errors': None, 'creationTime': '2021-10-28T14:08:02.758890+00:00', 'modifiedTime': '2021-10-28T14:08:18.269480+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 2, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC24S_V3'}


In [21]:
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import GridParameterSampling, choice
from azureml.automl.core.shared.constants import ImageTask

arguments = ["--early_stopping", 1, "--evaluation_frequency", 2]

automl_image_config_yolov5 = AutoMLImageConfig(task=ImageTask.IMAGE_OBJECT_DETECTION,
                                               compute_target=compute_target,
                                               training_data=dataset,
                                               validation_data=dataset,
                                               hyperparameter_sampling=GridParameterSampling({'model_name': choice('yolov5')}),
                                               #primary_metric='mean_average_precision',
                                               iterations=1)

In [23]:
ws = Workspace.from_config()
experiment = Experiment(ws, "coco128-automl-image-object-detection")
automl_image_run = experiment.submit(automl_image_config_yolov5)
automl_image_run.wait_for_completion(wait_post_processing=True)

Class SynapseCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
coco128-automl-image-object-detection,AutoML_035ab74b-895e-45d6-8431-b5161a7b11aa,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


{'runId': 'AutoML_035ab74b-895e-45d6-8431-b5161a7b11aa',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-24T18:18:22.149576Z',
 'endTimeUtc': '2022-04-24T18:32:03.215667Z',
 'services': {},
 'properties': {'num_iterations': '1',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'mean_average_precision',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'gpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"coco128-automl-image-object-detection","subscription_id":"c46a9435-c957-4e6c-a0f4-b9a597984773","resource_group":"mlops","workspace_name":"mlopsdev","region":"centralus","compute_target":"gpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1,"primary_metric":

In [24]:
best_child_run = automl_image_run.get_best_child()
model_name = best_child_run.properties['model_name']
model = best_child_run.register_model(model_name = model_name, model_path='outputs/model.pt')

In [25]:
model


Model(workspace=Workspace.create(name='mlopsdev', subscription_id='c46a9435-c957-4e6c-a0f4-b9a597984773', resource_group='mlops'), name=AutoML035ab74b80, id=AutoML035ab74b80:1, version=1, tags={}, properties={})

In [26]:
from azureml.automl.core.shared.constants import ImageTask
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import BanditPolicy, RandomParameterSampling
from azureml.train.hyperdrive import choice, uniform

parameter_space = {
    "model": choice(
        {
            "model_name": choice("yolov5"),
            "learning_rate": uniform(0.0001, 0.01),
            "model_size": choice("small", "medium"),  # model-specific
            #'img_size': choice(640, 704, 768), # model-specific; might need GPU with large memory
        },
        {
            "model_name": choice("fasterrcnn_resnet50_fpn"),
            "learning_rate": uniform(0.0001, 0.001),
            "optimizer": choice("sgd", "adam", "adamw"),
            "min_size": choice(600, 800),  # model-specific
            #'warmup_cosine_lr_warmup_epochs': choice(0, 3),
        },
    ),
}

tuning_settings = {
    "iterations": 2,
    "max_concurrent_iterations": 2,
    "hyperparameter_sampling": RandomParameterSampling(parameter_space),
    "early_termination_policy": BanditPolicy(
        evaluation_interval=2, slack_factor=0.2, delay_evaluation=6
    ),
}

automl_image_config = AutoMLImageConfig(
    task=ImageTask.IMAGE_OBJECT_DETECTION,
    compute_target=compute_target,
    training_data=dataset,
    validation_data=dataset,
    **tuning_settings,
)

In [ ]:
automl_image_run = experiment.submit(automl_image_config)
automl_image_run.wait_for_completion(wait_post_processing=True)